# Perceptron From Scratch — 4 Problem-Specific Implementations

This notebook implements and tests four perceptron variants **from scratch** (NumPy only):

1) **Binary Perceptron** (linearly separable binary classification)  
2) **Pocket Perceptron** (binary classification with non-separable/noisy data)  
3) **Kernel Perceptron (RBF)** (nonlinear separations like XOR)  
4) **Multiclass Perceptron (joint)** (K-way linear classification)

Each section includes:
- the node structure (minimal diagram),
- what layers exist,
- what structural change solves which problem,
- an implementation,
- experiments and plots.


In [3]:
import torch 
torch.cuda.is_available()

: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

def train_test_split(X, y, test_size=0.25, shuffle=True, seed=0):
    n = X.shape[0]
    idx = np.arange(n)
    if shuffle:
        rng = np.random.default_rng(seed)
        rng.shuffle(idx)
    cut = int(n * (1 - test_size))
    train_idx, test_idx = idx[:cut], idx[cut:]
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

def standardize_fit(X):
    mu = X.mean(axis=0)
    sigma = X.std(axis=0)
    sigma[sigma == 0] = 1.0
    return mu, sigma

def standardize_apply(X, mu, sigma):
    return (X - mu) / sigma

def accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def plot_2d_points(X, y, title="", ax=None):
    if ax is None:
        ax = plt.gca()
    classes = np.unique(y)
    for c in classes:
        m = (y == c)
        ax.scatter(X[m, 0], X[m, 1], s=25, label=str(c), alpha=0.9)
    ax.set_title(title)
    ax.legend()

def plot_decision_regions_2d(model, X, y, title="", ax=None, grid_step=0.03):
    if ax is None:
        ax = plt.gca()
    x_min, x_max = X[:, 0].min() - 0.8, X[:, 0].max() + 0.8
    y_min, y_max = X[:, 1].min() - 0.8, X[:, 1].max() + 0.8
    xs = np.arange(x_min, x_max, grid_step)
    ys = np.arange(y_min, y_max, grid_step)
    xx, yy = np.meshgrid(xs, ys)
    grid = np.c_[xx.ravel(), yy.ravel()]
    preds = model.predict(grid).reshape(xx.shape)
    ax.contourf(xx, yy, preds, alpha=0.25, levels=np.unique(preds).size)
    plot_2d_points(X, y, title=title, ax=ax)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)


## 1) Binary Perceptron

### Simplest node structure
```
x1 ─┐
x2 ─┼─> [ s = Σ wi*xi + b ] ─> [ sign ] ─> ŷ ∈ {−1,+1}
...─┤
xd ─┘
```

### Layers
- **Layer 1:** linear score: `s = w·x + b`
- **Layer 2:** hard decision: `ŷ = sign(s)`

### What problem it targets
- Fixed-length vectors → **binary classification** → **one linear boundary** (a hyperplane).

### Training rule (core idea)
- Only update when the current example is misclassified:
  - if `y * (w·x + b) <= 0`:
    - `w ← w + η y x`
    - `b ← b + η y`


In [ ]:
class BinaryPerceptron:
    def __init__(self, lr=1.0, epochs=50, shuffle=True, seed=0, use_bias=True):
        self.lr = lr
        self.epochs = epochs
        self.shuffle = shuffle
        self.seed = seed
        self.use_bias = use_bias

    def fit(self, X, y):
        # y must be in {-1, +1}
        y = y.astype(int)
        assert set(np.unique(y)).issubset({-1, 1})

        n, d = X.shape
        self.w = np.zeros(d, dtype=float)
        self.b = 0.0

        rng = np.random.default_rng(self.seed)
        self.mistakes_per_epoch = []

        for ep in range(self.epochs):
            idx = np.arange(n)
            if self.shuffle:
                rng.shuffle(idx)

            mistakes = 0
            for i in idx:
                s = float(np.dot(self.w, X[i]) + (self.b if self.use_bias else 0.0))
                if y[i] * s <= 0:
                    self.w += self.lr * y[i] * X[i]
                    if self.use_bias:
                        self.b += self.lr * y[i]
                    mistakes += 1

            self.mistakes_per_epoch.append(mistakes)

            # separable case early stop: an epoch with 0 mistakes
            if mistakes == 0:
                break

        return self

    def decision_function(self, X):
        return X @ self.w + (self.b if self.use_bias else 0.0)

    def predict(self, X):
        s = self.decision_function(X)
        # tie goes to +1
        return np.where(s >= 0, 1, -1)


### Experiment: linearly separable 2D data

We generate two clusters with a clear gap (separable), train the binary perceptron,
and plot the decision regions and mistake count per epoch.


In [ ]:
def make_separable_blobs(n=200, gap=2.5, std=0.6, seed=0):
    rng = np.random.default_rng(seed)
    n1 = n // 2
    n2 = n - n1
    X1 = rng.normal(loc=(-gap, -gap), scale=std, size=(n1, 2))
    X2 = rng.normal(loc=( gap,  gap), scale=std, size=(n2, 2))
    X = np.vstack([X1, X2])
    y = np.hstack([np.full(n1, -1), np.full(n2, +1)])
    return X, y

X, y = make_separable_blobs(n=240, gap=2.2, std=0.7, seed=1)

# standardize (helps convergence)
mu, sigma = standardize_fit(X)
Xs = standardize_apply(X, mu, sigma)

Xtr, Xte, ytr, yte = train_test_split(Xs, y, test_size=0.25, seed=1)

model = BinaryPerceptron(lr=1.0, epochs=50, seed=1)
model.fit(Xtr, ytr)

print("Train acc:", accuracy(ytr, model.predict(Xtr)))
print("Test  acc:", accuracy(yte, model.predict(Xte)))
print("Epochs run:", len(model.mistakes_per_epoch))
print("Mistakes per epoch:", model.mistakes_per_epoch)

fig, ax = plt.subplots(1, 2, figsize=(12, 4))
plot_decision_regions_2d(model, Xte, yte, title="Binary Perceptron decision regions (test)", ax=ax[0])
ax[1].plot(model.mistakes_per_epoch, marker="o")
ax[1].set_title("Mistakes per epoch")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Mistakes")
plt.show()


## 2) Pocket Perceptron (non-separable / noisy)

### Node structure
Same as the binary perceptron (no structural change in the graph).

### What changes (to solve a different problem)
The standard perceptron may never converge on non-separable data.
**Pocket** keeps the **best weights seen so far** (best validation accuracy),
so you do not end up with a bad final iterate.

### One-sentence training idea
- Keep a “pocket” copy of `(w,b)` whenever it improves validation accuracy.


In [ ]:
class PocketPerceptron:
    def __init__(self, lr=1.0, epochs=100, shuffle=True, seed=0, use_bias=True, val_split=0.25):
        self.lr = lr
        self.epochs = epochs
        self.shuffle = shuffle
        self.seed = seed
        self.use_bias = use_bias
        self.val_split = val_split

    def fit(self, X, y):
        y = y.astype(int)
        assert set(np.unique(y)).issubset({-1, 1})

        # internal split for pocket evaluation
        Xtr, Xval, ytr, yval = train_test_split(X, y, test_size=self.val_split, seed=self.seed)

        n, d = Xtr.shape
        self.w = np.zeros(d, dtype=float)
        self.b = 0.0

        # pocket best
        best_w = self.w.copy()
        best_b = self.b
        best_acc = -1.0

        rng = np.random.default_rng(self.seed)
        self.val_acc_per_epoch = []
        self.train_mistakes_per_epoch = []

        for ep in range(self.epochs):
            idx = np.arange(n)
            if self.shuffle:
                rng.shuffle(idx)

            mistakes = 0
            for i in idx:
                s = float(np.dot(self.w, Xtr[i]) + (self.b if self.use_bias else 0.0))
                if ytr[i] * s <= 0:
                    self.w += self.lr * ytr[i] * Xtr[i]
                    if self.use_bias:
                        self.b += self.lr * ytr[i]
                    mistakes += 1

            self.train_mistakes_per_epoch.append(mistakes)

            # evaluate on validation
            val_pred = self.predict(Xval)
            val_acc = accuracy(yval, val_pred)
            self.val_acc_per_epoch.append(val_acc)

            if val_acc > best_acc:
                best_acc = val_acc
                best_w = self.w.copy()
                best_b = self.b

        # return best
        self.w = best_w
        self.b = best_b
        self.best_val_acc = best_acc
        return self

    def decision_function(self, X):
        return X @ self.w + (self.b if self.use_bias else 0.0)

    def predict(self, X):
        s = self.decision_function(X)
        return np.where(s >= 0, 1, -1)


### Experiment: noisy binary data (not separable)

We create overlapping classes and flip some labels.  
Compare **standard** vs **pocket** on test accuracy.


In [ ]:
def make_noisy_overlap(n=300, std=1.4, label_flip=0.15, seed=0):
    rng = np.random.default_rng(seed)
    n1 = n // 2
    n2 = n - n1
    X1 = rng.normal(loc=(-1.0, -1.0), scale=std, size=(n1, 2))
    X2 = rng.normal(loc=( 1.0,  1.0), scale=std, size=(n2, 2))
    X = np.vstack([X1, X2])
    y = np.hstack([np.full(n1, -1), np.full(n2, +1)])

    # flip labels
    m = int(label_flip * n)
    flip_idx = rng.choice(n, size=m, replace=False)
    y[flip_idx] *= -1
    return X, y

X, y = make_noisy_overlap(n=400, std=1.5, label_flip=0.18, seed=2)
mu, sigma = standardize_fit(X)
Xs = standardize_apply(X, mu, sigma)

Xtr, Xte, ytr, yte = train_test_split(Xs, y, test_size=0.3, seed=2)

std_model = BinaryPerceptron(lr=1.0, epochs=60, seed=2)
std_model.fit(Xtr, ytr)

pocket_model = PocketPerceptron(lr=1.0, epochs=60, seed=2, val_split=0.25)
pocket_model.fit(Xtr, ytr)

print("Standard perceptron test acc:", accuracy(yte, std_model.predict(Xte)))
print("Pocket perceptron   test acc:", accuracy(yte, pocket_model.predict(Xte)))
print("Pocket best val acc:", pocket_model.best_val_acc)

fig, ax = plt.subplots(1, 3, figsize=(15, 4))
plot_decision_regions_2d(std_model, Xte, yte, title="Standard perceptron (test)", ax=ax[0])
plot_decision_regions_2d(pocket_model, Xte, yte, title="Pocket perceptron (test)", ax=ax[1])
ax[2].plot(pocket_model.val_acc_per_epoch, marker="o", alpha=0.8)
ax[2].set_title("Pocket: validation accuracy per epoch")
ax[2].set_xlabel("Epoch")
ax[2].set_ylabel("Val accuracy")
plt.show()


## 3) Kernel Perceptron (RBF) — nonlinear separation (XOR)

### Node structure
Same final decision node, but the “linear score” is replaced by a **kernel score**:

```
stored examples (xi, yi, αi)  ─┐
                              ├─> s(x) = Σ αi yi K(xi, x) + b ─> sign ─> ŷ
current x --------------------┘
```

### What changes (to solve a different problem)
A linear perceptron cannot solve XOR in the original input space.  
Kernel perceptron replaces `x·x'` with `K(x, x')`, which acts like a nonlinear feature map.

### One-sentence training idea
- Keep coefficients `αi`; on a mistake, increase the coefficient for that training example.


In [ ]:
def rbf_kernel(X1, X2, gamma=1.0):
    # X1: (n1,d), X2: (n2,d)
    # returns K: (n1,n2)
    # K(x,z) = exp(-gamma * ||x-z||^2)
    X1_sq = np.sum(X1**2, axis=1, keepdims=True)
    X2_sq = np.sum(X2**2, axis=1, keepdims=True).T
    dist2 = X1_sq + X2_sq - 2 * (X1 @ X2.T)
    return np.exp(-gamma * dist2)

class KernelPerceptronRBF:
    def __init__(self, gamma=1.0, lr=1.0, epochs=30, shuffle=True, seed=0, use_bias=True):
        self.gamma = gamma
        self.lr = lr
        self.epochs = epochs
        self.shuffle = shuffle
        self.seed = seed
        self.use_bias = use_bias

    def fit(self, X, y):
        y = y.astype(int)
        assert set(np.unique(y)).issubset({-1, 1})

        self.X_train = X.copy()
        self.y_train = y.copy()
        n = X.shape[0]
        self.alpha = np.zeros(n, dtype=float)
        self.b = 0.0

        # precompute Gram matrix for speed on small datasets
        K = rbf_kernel(self.X_train, self.X_train, gamma=self.gamma)

        rng = np.random.default_rng(self.seed)
        self.mistakes_per_epoch = []

        for ep in range(self.epochs):
            idx = np.arange(n)
            if self.shuffle:
                rng.shuffle(idx)

            mistakes = 0
            for i in idx:
                # score using kernel expansion over training points
                s = np.sum(self.alpha * self.y_train * K[:, i]) + (self.b if self.use_bias else 0.0)
                if self.y_train[i] * s <= 0:
                    self.alpha[i] += self.lr
                    if self.use_bias:
                        self.b += self.lr * self.y_train[i]
                    mistakes += 1

            self.mistakes_per_epoch.append(mistakes)
            if mistakes == 0:
                break

        return self

    def decision_function(self, X):
        K = rbf_kernel(self.X_train, X, gamma=self.gamma)  # (n_train, n_test)
        s = (self.alpha * self.y_train) @ K
        if self.use_bias:
            s = s + self.b
        return s

    def predict(self, X):
        s = self.decision_function(X)
        return np.where(s >= 0, 1, -1)


### Experiment: XOR (nonlinear) in 2D

- Standard perceptron fails because XOR is not linearly separable.
- Kernel perceptron with RBF kernel can separate it.


In [ ]:
def make_xor(n=300, noise=0.25, seed=0):
    rng = np.random.default_rng(seed)
    X = rng.uniform(-1.5, 1.5, size=(n, 2))
    y = np.where(X[:, 0] * X[:, 1] >= 0, 1, -1)  # same sign => +1, different => -1
    X = X + rng.normal(scale=noise, size=X.shape)
    return X, y

X, y = make_xor(n=500, noise=0.18, seed=3)
mu, sigma = standardize_fit(X)
Xs = standardize_apply(X, mu, sigma)

Xtr, Xte, ytr, yte = train_test_split(Xs, y, test_size=0.3, seed=3)

linear = BinaryPerceptron(lr=1.0, epochs=50, seed=3)
linear.fit(Xtr, ytr)

kernel = KernelPerceptronRBF(gamma=2.0, lr=1.0, epochs=30, seed=3)
kernel.fit(Xtr, ytr)

print("Linear perceptron test acc:", accuracy(yte, linear.predict(Xte)))
print("Kernel  perceptron test acc:", accuracy(yte, kernel.predict(Xte)))

fig, ax = plt.subplots(1, 2, figsize=(12, 4))
plot_decision_regions_2d(linear, Xte, yte, title="Linear perceptron on XOR (fails)", ax=ax[0])
plot_decision_regions_2d(kernel, Xte, yte, title="Kernel perceptron (RBF) on XOR", ax=ax[1])
plt.show()

plt.figure(figsize=(6,4))
plt.plot(kernel.mistakes_per_epoch, marker="o")
plt.title("Kernel perceptron mistakes per epoch")
plt.xlabel("Epoch")
plt.ylabel("Mistakes")
plt.show()


## 4) Multiclass Perceptron (joint)

### Simplest node structure
Instead of one score node, we compute **K scores** and choose the largest:

```
x -> [ s1 = w1·x + b1 ]
x -> [ s2 = w2·x + b2 ]  -> argmax -> class
...
x -> [ sK = wK·x + bK ]
```

### Layers
- **Layer 1:** K linear score units (one per class)
- **Layer 2:** argmax decision

### What changes (to solve a different problem)
Binary perceptron can only choose between 2 classes.
Adding **K score nodes** and an **argmax** makes it work for K-way classification.

### One-sentence training idea
- If the predicted class is wrong, add `x` to the true class weights and subtract `x` from the predicted class weights.


In [ ]:
class MulticlassPerceptron:
    def __init__(self, lr=1.0, epochs=60, shuffle=True, seed=0, use_bias=True):
        self.lr = lr
        self.epochs = epochs
        self.shuffle = shuffle
        self.seed = seed
        self.use_bias = use_bias

    def fit(self, X, y):
        y = y.astype(int)
        classes = np.unique(y)
        self.classes_ = classes
        K = classes.size
        n, d = X.shape

        # map labels to 0..K-1 internally
        self.label_to_idx_ = {c:i for i,c in enumerate(classes)}
        self.idx_to_label_ = {i:c for i,c in enumerate(classes)}
        y_idx = np.array([self.label_to_idx_[c] for c in y], dtype=int)

        self.W = np.zeros((K, d), dtype=float)
        self.b = np.zeros(K, dtype=float)

        rng = np.random.default_rng(self.seed)
        self.mistakes_per_epoch = []

        for ep in range(self.epochs):
            idx = np.arange(n)
            if self.shuffle:
                rng.shuffle(idx)

            mistakes = 0
            for i in idx:
                scores = self.W @ X[i] + (self.b if self.use_bias else 0.0)
                pred = int(np.argmax(scores))
                true = int(y_idx[i])
                if pred != true:
                    self.W[true] += self.lr * X[i]
                    self.W[pred] -= self.lr * X[i]
                    if self.use_bias:
                        self.b[true] += self.lr
                        self.b[pred] -= self.lr
                    mistakes += 1

            self.mistakes_per_epoch.append(mistakes)
            if mistakes == 0:
                break

        return self

    def decision_function(self, X):
        return X @ self.W.T + (self.b if self.use_bias else 0.0)

    def predict(self, X):
        scores = self.decision_function(X)
        pred_idx = np.argmax(scores, axis=1)
        return np.array([self.idx_to_label_[int(i)] for i in pred_idx], dtype=int)


### Experiment: 3-class 2D blobs

We create three clusters and train the joint multiclass perceptron.


In [ ]:
def make_multiclass_blobs(n=450, std=0.8, seed=0):
    rng = np.random.default_rng(seed)
    centers = np.array([[-2.5, 0.0], [2.5, 0.0], [0.0, 2.8]])
    K = centers.shape[0]
    per = n // K
    Xs = []
    ys = []
    for k in range(K):
        Xk = rng.normal(loc=centers[k], scale=std, size=(per, 2))
        yk = np.full(per, k, dtype=int)
        Xs.append(Xk); ys.append(yk)
    X = np.vstack(Xs)
    y = np.hstack(ys)
    return X, y

X, y = make_multiclass_blobs(n=480, std=0.9, seed=4)
mu, sigma = standardize_fit(X)
Xs = standardize_apply(X, mu, sigma)

Xtr, Xte, ytr, yte = train_test_split(Xs, y, test_size=0.3, seed=4)

mc = MulticlassPerceptron(lr=1.0, epochs=80, seed=4)
mc.fit(Xtr, ytr)

print("Train acc:", accuracy(ytr, mc.predict(Xtr)))
print("Test  acc:", accuracy(yte, mc.predict(Xte)))
print("Epochs run:", len(mc.mistakes_per_epoch))

fig, ax = plt.subplots(1, 2, figsize=(12, 4))
plot_decision_regions_2d(mc, Xte, yte, title="Multiclass perceptron decision regions (test)", ax=ax[0])
ax[1].plot(mc.mistakes_per_epoch, marker="o")
ax[1].set_title("Mistakes per epoch")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Mistakes")
plt.show()


## Notes for education

- The **node structure** changes only when the **output format** changes (binary → multiclass) or when the **score computation** changes (linear → kernel).
- For noisy data, the **graph stays the same**; what changes is **which weights you keep** (pocket).

## Suggested exercises (optional)
1) Add a **margin perceptron** by updating when `y*s <= delta`.  
2) Add an **averaged perceptron** and compare it to pocket on noisy data.  
3) Try different RBF `gamma` on XOR and observe under/overfitting.
